## Filtra informe diário APENAS para os fundos  filtrados a seguir:
- ### Multimercados
- ### Em funcionamento normal
- ### NÃO Exclusivos
- ### Com mais de 1 cotista


## Monica Barros

## Data: 19/02/2021

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

plt.style.use('fivethirtyeight')
pd.set_option('max_colwidth', 200)  # Altera largura máxima das colunas
pd.set_option('display.max_colwidth', None)

import seaborn as sns; 
sns.set(rc={'figure.figsize':(17,6)})

plt.rcParams['axes.labelsize'] = 16
plt.rcParams['xtick.labelsize'] = 14
plt.rcParams['ytick.labelsize'] = 14

import os
import datetime
from dateutil.relativedelta import relativedelta
import time
from time import perf_counter

import glob
import pickle

#!pip install openpyxl
from openpyxl import Workbook

#!pip install pytz
import pytz
local_tz = pytz.timezone('America/Sao_Paulo')

In [2]:
dir_cadastro =  '/home/jovyan/work/@Fund_Eval/CVM/Cadastro/'
dir_cot_fundos = '/home/jovyan/work/@Fund_Eval/CVM/Diario/'
dir_fat_risco = '/home/jovyan/work/@Fund_Eval/@fatores_risco/'

In [3]:
cadastro_mm = pd.read_pickle(dir_cadastro + 'cadastro_fundos_multimercado_ready_to_go_20210219.pkl')

In [4]:
cadastro_mm.shape

(62689, 36)

In [5]:
cadastro_mm.columns


Index(['CNPJ_FUNDO', 'DENOM_SOCIAL', 'DT_REG', 'DT_CONST', 'DT_CANCEL', 'SIT',
       'DT_INI_SIT', 'DT_INI_ATIV', 'DT_INI_EXERC', 'DT_FIM_EXERC', 'CLASSE',
       'DT_INI_CLASSE', 'RENTAB_FUNDO', 'CONDOM', 'FUNDO_COTAS',
       'FUNDO_EXCLUSIVO', 'TRIB_LPRAZO', 'INVEST_QUALIF', 'TAXA_PERFM',
       'VL_PATRIM_LIQ', 'DT_PATRIM_LIQ', 'DIRETOR', 'CNPJ_ADMIN', 'ADMIN',
       'PF_PJ_GESTOR', 'CPF_CNPJ_GESTOR', 'GESTOR', 'CNPJ_AUDITOR', 'AUDITOR',
       'INF_TAXA_PERFM', 'CNPJ_CUSTODIANTE', 'CUSTODIANTE', 'CNPJ_CONTROLADOR',
       'CONTROLADOR', 'TAXA_ADM', 'INF_TAXA_ADM'],
      dtype='object')

In [6]:
fundos_mm = pd.read_pickle(dir_cot_fundos + 'df_diario_2017_202102.pkl')

In [7]:
fundos_mm.shape

(15982449, 8)

In [8]:
fundos_mm.columns

Index(['CNPJ_FUNDO', 'DT_COMPTC', 'VL_TOTAL', 'VL_QUOTA', 'VL_PATRIM_LIQ',
       'CAPTC_DIA', 'RESG_DIA', 'NR_COTST'],
      dtype='object')

In [9]:
fundos_mm.NR_COTST.value_counts().to_frame()

,NR_COTST
1,7571197
2,1488268
3,865050
4,624206
5,369182
...,...
45042,1
155163,1
126938,1
94154,1


### Separa fundos com só 1 cotista e os com mais de 1 cotista

In [10]:
fundos_mm_1_quot = fundos_mm[fundos_mm['NR_COTST']==1]
fundos_mm_1_quot.shape

(7571197, 8)

In [11]:
# Número de fundos MM com mais de 1 cotista é 15982449-7571197 = 8411252, um pouco maior que os com só 1 cotista

In [12]:
fundos_mm_1_quot.head()

,CNPJ_FUNDO,DT_COMPTC,VL_TOTAL,VL_QUOTA,VL_PATRIM_LIQ,CAPTC_DIA,RESG_DIA,NR_COTST
0,00.017.024/0001-53,2017-01-02,1082310.35,24.264751,1080998.58,0.0,0.0,1
1,00.017.024/0001-53,2017-01-03,1082843.72,24.274863,1081449.09,0.0,0.0,1
2,00.017.024/0001-53,2017-01-04,1083398.28,24.284681,1081886.49,0.0,0.0,1
3,00.017.024/0001-53,2017-01-05,1083940.08,24.294975,1082345.09,0.0,0.0,1
4,00.017.024/0001-53,2017-01-06,1084174.96,24.304952,1082789.54,0.0,0.0,1


# Será que os fundos com 1 cotista são os fundos Master?

## Não necessariamente, ao que parece, olhando o cadastro.

In [13]:
tt1 = cadastro_mm[['CNPJ_FUNDO','DENOM_SOCIAL']].merge(fundos_mm_1_quot[['CNPJ_FUNDO']], on ='CNPJ_FUNDO')
tt1 = tt1.drop_duplicates()
tt1 = tt1.sort_values(by = 'CNPJ_FUNDO', ascending=True)
tt1 = tt1.drop_duplicates()

In [14]:
tt1.shape

(7030, 2)

In [15]:
tt1.head(15)

,CNPJ_FUNDO,DENOM_SOCIAL
3546975,00.346.750/0001-10,NIC YIELD FUNDO DE INVESTIMENTO EM COTAS DE FUNDOS DE INVESTIMENTO MULTIMERCADO
3546965,00.346.750/0001-10,COINVALORES YIELD FUNDO DE INVESTIMENTO EM COTAS DE FUNDOS DE INVESTIMENTO MULTIMERCADO
17862040,00.754.543/0001-02,TARGET FUNDO DE INVESTIMENTO EM COTAS DE FUNDO DE INVESTIMENTO MULTIMERCADO CRÉDITO PRIVADO
9585792,00.833.458/0001-21,FUNDO DE INVESTIMENTO MULTIMERCADO TRIUNFO
12749590,00.836.255/0001-99,MARACUJÁ FUNDO DE INVESTIMENTO MULTIMERCADO CRÉDITO PRIVADO - INVESTIMENTO NO EXTERIOR
3439965,00.846.898/0001-13,CHEMICAL FUNDO DE INVESTIMENTO MULTIMERCADO CREDITO PRIVADO INVESTIMENTO NO EXTERIOR
7652462,00.893.531/0001-50,FAPA SENIOR FUNDO DE INVESTIMENTO MULTIMERCADO
2318059,01.392.020/0001-18,BRADESCO FUNDO DE INVESTIMENTO EM COTAS DE FI MULTIMERCADO PGBL/VGBL FUTURE COMPOSTO III
14270609,01.452.621/0001-79,PB II FICFI MULTIMERCADO CRÉDITO PRIVADO IE
20301221,01.562.963/0001-41,FCOPEL FUNDO DE INVESTIMENTO MULTIMERCADO IMA-B II


# Fundos multimercado com mais de um cotista

In [16]:
fundos_mm_1plus_quot = fundos_mm[fundos_mm['NR_COTST'] > 1]
fundos_mm_1plus_quot.shape

(8345062, 8)

In [17]:
tt2 = cadastro_mm[['CNPJ_FUNDO','DENOM_SOCIAL']].merge(fundos_mm_1plus_quot[['CNPJ_FUNDO']], on ='CNPJ_FUNDO')
tt2 = tt2.drop_duplicates()
tt2 = tt2.sort_values(by = 'CNPJ_FUNDO', ascending=True)
tt2 = tt2.drop_duplicates()

In [18]:
tt2.shape

(7567, 2)

In [19]:
tt2.head(15)

,CNPJ_FUNDO,DENOM_SOCIAL
6234354,00.346.750/0001-10,COINVALORES YIELD FUNDO DE INVESTIMENTO EM COTAS DE FUNDOS DE INVESTIMENTO MULTIMERCADO
6236398,00.346.750/0001-10,NIC YIELD FUNDO DE INVESTIMENTO EM COTAS DE FUNDOS DE INVESTIMENTO MULTIMERCADO
20875931,00.400.490/0001-13,PORTO SEGURO MACRO FUNDO DE INVESTIMENTO EM COTAS DE FUNDOS DE INVESTIMENTO MULTIMERCADO
13611237,00.539.553/0001-17,GUIDE CASH FUNDO DE INVESTIMENTO EM COTAS DE FUNDOS DE INVESTIMENTO MULTIMERCADO CRÉDITO PRIVADO
9054458,00.826.903/0001-26,CSHG TOP FUNDO DE INVESTIMENTO EM COTAS DE FUNDOS DE INVESTIMENTO MULTIMERCADO
4185450,00.829.163/0001-81,BRADESCO FUNDO DE INVESTIMENTO EM COTAS DE FUNDOS DE INVESTIMENTO MULTIMERCADO MULTIPERFORMANCE
12425828,00.833.458/0001-21,FUNDO DE INVESTIMENTO MULTIMERCADO TRIUNFO
18173990,00.836.255/0001-99,MARACUJÁ FUNDO DE INVESTIMENTO MULTIMERCADO CRÉDITO PRIVADO - INVESTIMENTO NO EXTERIOR
15464392,00.838.868/0001-65,ITAÚ PRIVATE SPECIAL MULTIMERCADO CRÉDITO PRIVADO FUNDO DE INVESTIMENTO
15116839,00.840.008/0001-66,ITAÚ MULTIMERCADO MULTIMANAGER FUNDO DE INVESTIMENTO EM COTAS DE FUNDOS DE INVESTIMENTO


#  Criando dataframe "pequena" para teste - apenas fundos com mais de um cotista

## O sample tem que ser feito sobre a coluna de CNPJs, e não sobre a dataframe inteira

In [20]:
sample_cnpj= fundos_mm_1plus_quot.CNPJ_FUNDO.sample(frac = 0.001) ##iloc[:20000,:]
sample_cnpj.shape

(8345,)

In [21]:
#Cria lista com cnpjs amostrados
l_sample_cnpj = sample_cnpj.to_list()

In [22]:
small_1plus_quot = fundos_mm_1plus_quot[fundos_mm_1plus_quot['CNPJ_FUNDO'].isin(l_sample_cnpj)]
small_1plus_quot.shape

(4659816, 8)

In [23]:
small_1plus_quot.CNPJ_FUNDO.value_counts()

04.895.190/0001-03    1033
14.097.811/0001-67    1033
24.038.672/0001-54    1033
03.919.892/0001-17    1033
09.577.036/0001-07    1033
                      ... 
15.453.091/0001-98      27
22.504.024/0001-10      23
39.723.636/0001-05      23
21.082.091/0001-21      23
37.487.547/0001-73      11
Name: CNPJ_FUNDO, Length: 5689, dtype: int64

## Dataframe "pequeno" contendo CNPJ, Valor Quota, Patrim. Liquido e Número de Cotistas - apenas fundos com + de 1 cotista

In [30]:
uu = small_1plus_quot.pivot_table(index="DT_COMPTC", columns='CNPJ_FUNDO', values = ['VL_QUOTA', 'VL_PATRIM_LIQ','NR_COTST'])#, values="quantity", aggfunc=[np.sum, np.mean], fill_value=0)

#fundos = fundos.pivot(index='DT_COMPTC', columns='CNPJ_FUNDO')  
#cotas_normalizadas = fundos['VL_QUOTA'] / fundos['VL_QUOTA'].iloc[0]

In [31]:
uu.shape

(1033, 17067)

In [32]:
uu.iloc[:,0:4].columns

MultiIndex([('NR_COTST', '00.071.477/0001-68'),
            ('NR_COTST', '00.073.041/0001-08'),
            ('NR_COTST', '00.083.181/0001-67'),
            ('NR_COTST', '00.180.995/0001-10')],
           names=[None, 'CNPJ_FUNDO'])

In [33]:
uu = uu.reset_index(level=[0])
uu.iloc[:,0:4].columns

MultiIndex([('DT_COMPTC',                   ''),
            ( 'NR_COTST', '00.071.477/0001-68'),
            ( 'NR_COTST', '00.073.041/0001-08'),
            ( 'NR_COTST', '00.083.181/0001-67')],
           names=[None, 'CNPJ_FUNDO'])

In [34]:
uu = uu.set_index('DT_COMPTC')

In [35]:
uu.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1033 entries, 2017-01-02 to 2021-02-12
Columns: 17067 entries, ('NR_COTST', '00.071.477/0001-68') to ('VL_QUOTA', '97.929.213/0001-34')
dtypes: float64(17067)
memory usage: 134.5+ MB


In [36]:
uu5 = uu.copy()  # Criando Backup

In [37]:
uu.columns = [' '.join(col).strip() for col in uu.columns.values]
uu.head()

,NR_COTST 00.071.477/0001-68,NR_COTST 00.073.041/0001-08,NR_COTST 00.083.181/0001-67,NR_COTST 00.180.995/0001-10,NR_COTST 00.185.259/0001-54,NR_COTST 00.194.256/0001-87,NR_COTST 00.211.294/0001-09,NR_COTST 00.222.725/0001-24,NR_COTST 00.222.816/0001-60,NR_COTST 00.306.278/0001-91,...,VL_QUOTA 74.326.471/0001-20,VL_QUOTA 88.002.696/0001-36,VL_QUOTA 96.498.985/0001-04,VL_QUOTA 97.261.093/0001-40,VL_QUOTA 97.519.573/0001-68,VL_QUOTA 97.525.430/0001-69,VL_QUOTA 97.543.707/0001-86,VL_QUOTA 97.548.167/0001-23,VL_QUOTA 97.929.197/0001-80,VL_QUOTA 97.929.213/0001-34
DT_COMPTC,,,,,,,,,,,,,,,,,,,,,
2017-01-02,127848.0,1228.0,7.0,3371.0,217.0,52.0,NaN,3337.0,7676.0,6490.0,...,9.158518,0.408780,13.593847,8.54950,18.622646,1486.778966,1.058470,2.132760,2.439919,7.396478
2017-01-03,128903.0,1227.0,7.0,3376.0,217.0,52.0,NaN,3337.0,7682.0,6497.0,...,9.124763,0.423409,14.099995,8.86087,18.735179,1487.623452,1.097389,2.131012,2.463890,7.403134
2017-01-04,129057.0,1227.0,7.0,3378.0,217.0,52.0,NaN,3335.0,7689.0,6504.0,...,9.078323,0.421930,14.046469,8.85104,18.750355,1488.408886,1.097418,2.128621,2.463123,7.384956
2017-01-05,128008.0,1227.0,7.0,3367.0,216.0,52.0,NaN,3330.0,7695.0,6503.0,...,9.068999,0.425041,14.155190,8.93599,18.790564,1489.585369,1.102948,2.134223,2.466688,7.374894
2017-01-06,127970.0,1227.0,7.0,3369.0,215.0,52.0,NaN,3328.0,7703.0,6514.0,...,9.103039,0.422201,14.060769,8.85431,18.792315,1490.144850,1.099303,2.138853,2.463503,7.376661


In [38]:
uu.tail(3)

,NR_COTST 00.071.477/0001-68,NR_COTST 00.073.041/0001-08,NR_COTST 00.083.181/0001-67,NR_COTST 00.180.995/0001-10,NR_COTST 00.185.259/0001-54,NR_COTST 00.194.256/0001-87,NR_COTST 00.211.294/0001-09,NR_COTST 00.222.725/0001-24,NR_COTST 00.222.816/0001-60,NR_COTST 00.306.278/0001-91,...,VL_QUOTA 74.326.471/0001-20,VL_QUOTA 88.002.696/0001-36,VL_QUOTA 96.498.985/0001-04,VL_QUOTA 97.261.093/0001-40,VL_QUOTA 97.519.573/0001-68,VL_QUOTA 97.525.430/0001-69,VL_QUOTA 97.543.707/0001-86,VL_QUOTA 97.548.167/0001-23,VL_QUOTA 97.929.197/0001-80,VL_QUOTA 97.929.213/0001-34
DT_COMPTC,,,,,,,,,,,,,,,,,,,,,
2021-02-10,305658.0,943.0,4.0,2731.0,196.0,40.0,9.0,1490.0,6355.0,10120.0,...,20.834616,0.635982,24.139798,15.09552,26.856290,2272.313422,2.096676,3.537824,6.451052,9.535098
2021-02-11,308296.0,943.0,4.0,2711.0,196.0,40.0,9.0,1489.0,6338.0,10115.0,...,20.735484,0.642173,24.315172,15.16390,26.915484,2288.293174,2.114689,3.536515,6.541891,9.566748
2021-02-12,310006.0,942.0,4.0,2709.0,196.0,40.0,9.0,1489.0,6335.0,10113.0,...,20.750691,0.643280,24.340581,15.19843,26.927856,2291.690487,2.117458,3.537408,6.547986,9.585352


In [42]:
uu.iloc[:,11378:11390].head()

,VL_QUOTA 00.071.477/0001-68,VL_QUOTA 00.073.041/0001-08,VL_QUOTA 00.083.181/0001-67,VL_QUOTA 00.180.995/0001-10,VL_QUOTA 00.185.259/0001-54,VL_QUOTA 00.194.256/0001-87,VL_QUOTA 00.211.294/0001-09,VL_QUOTA 00.222.725/0001-24,VL_QUOTA 00.222.816/0001-60,VL_QUOTA 00.306.278/0001-91,VL_QUOTA 00.323.513/0001-33,VL_QUOTA 00.346.750/0001-10
DT_COMPTC,,,,,,,,,,,,
2017-01-02,8.979568,23.295499,265.575794,445.069343,26.092243,36.832725,NaN,3116.171986,6123.178038,15.470810,35.272547,26.775561
2017-01-03,8.982675,23.307125,270.351738,445.251432,26.547772,36.850714,NaN,3117.564552,6125.729102,15.478332,35.290219,26.794680
2017-01-04,8.985907,23.318703,270.139135,445.273699,26.539245,36.863293,NaN,3118.252874,6126.171733,15.485848,35.307467,26.803474
2017-01-05,8.989098,23.330050,271.678420,445.650770,26.686670,36.902384,NaN,3120.458618,6131.360943,15.493379,35.324739,26.823828
2017-01-06,8.992318,23.341399,271.179390,445.835990,26.634562,36.919236,NaN,3121.450084,6134.080987,15.501003,35.342011,26.840264


In [41]:
+2*17067/3

11378.0

## Salva em csv e em pkl a dataframe "pequena"

In [45]:
time_start = perf_counter()

uu.to_pickle(dir_fat_risco + 'small_fundos_mm_ready_to_go_20210223.pkl') 
uu.to_csv(dir_fat_risco + 'small_fundos_mm_ready_to_go_20210223.csv',sep = ';' ,encoding='utf-8') 

time_end = perf_counter()
elapsed_time = time_end - time_start
print(' ****salvar base "pequena" de fundos MM com mais de 1 cotista demorou', round(elapsed_time,2) , ' segundos *****')

 ****salvar base "pequena" de fundos MM com mais de 1 cotista 69.69  segundos *****


# Pivot da base de fundos COMPLETA

In [46]:
time_start = perf_counter()
vv = fundos_mm_1plus_quot.pivot_table(index="DT_COMPTC", columns='CNPJ_FUNDO', values = ['VL_QUOTA', 'VL_PATRIM_LIQ','NR_COTST'])#, values="quantity", aggfunc=[np.sum, np.mean], fill_value=0)
time_end = perf_counter()
elapsed_time = time_end - time_start
print(' **** Pivot da base COMPLETA de fundos MM com mais de 1 cotista demorou', round(elapsed_time,2) , ' segundos *****')


 ****Pivot da base COMPLETA de fundos MM com mais de 1 cotista demorou 15.65  segundos *****


In [49]:
vv.shape

(1033, 41067)

In [55]:
vv = vv.reset_index(level=[0])
print(vv.iloc[:,0:4].columns)
print('')
vv = vv.set_index('DT_COMPTC')
vv.columns = [' '.join(col).strip() for col in vv.columns.values]
display(vv.head(3))
print('')
display(vv.tail(3))

MultiIndex([('DT_COMPTC',                   ''),
            ( 'NR_COTST', '00.068.305/0001-35'),
            ( 'NR_COTST', '00.071.477/0001-68'),
            ( 'NR_COTST', '00.073.041/0001-08')],
           names=[None, 'CNPJ_FUNDO'])



,NR_COTST 00.068.305/0001-35,NR_COTST 00.071.477/0001-68,NR_COTST 00.073.041/0001-08,NR_COTST 00.083.181/0001-67,NR_COTST 00.089.915/0001-15,NR_COTST 00.102.322/0001-41,NR_COTST 00.129.444/0001-21,NR_COTST 00.180.995/0001-10,NR_COTST 00.185.259/0001-54,NR_COTST 00.194.256/0001-87,...,VL_QUOTA 97.525.430/0001-69,VL_QUOTA 97.525.452/0001-29,VL_QUOTA 97.525.454/0001-18,VL_QUOTA 97.543.626/0001-86,VL_QUOTA 97.543.707/0001-86,VL_QUOTA 97.548.162/0001-09,VL_QUOTA 97.548.167/0001-23,VL_QUOTA 97.711.812/0001-87,VL_QUOTA 97.929.197/0001-80,VL_QUOTA 97.929.213/0001-34
DT_COMPTC,,,,,,,,,,,,,,,,,,,,,
2017-01-02,8707.0,127848.0,1228.0,7.0,192.0,4.0,4.0,3371.0,217.0,52.0,...,1486.778966,2109.170473,172.376222,NaN,1.058470,2.304889,2.132760,2.491731,2.439919,7.396478
2017-01-03,8705.0,128903.0,1227.0,7.0,196.0,4.0,4.0,3376.0,217.0,52.0,...,1487.623452,2110.474066,172.514624,NaN,1.097389,2.286750,2.131012,2.536513,2.463890,7.403134
2017-01-04,8705.0,129057.0,1227.0,7.0,193.0,4.0,4.0,3378.0,217.0,52.0,...,1488.408886,2111.778259,172.278389,NaN,1.097418,2.266704,2.128621,2.536816,2.463123,7.384956


,NR_COTST 00.068.305/0001-35,NR_COTST 00.071.477/0001-68,NR_COTST 00.073.041/0001-08,NR_COTST 00.083.181/0001-67,NR_COTST 00.089.915/0001-15,NR_COTST 00.102.322/0001-41,NR_COTST 00.129.444/0001-21,NR_COTST 00.180.995/0001-10,NR_COTST 00.185.259/0001-54,NR_COTST 00.194.256/0001-87,...,VL_QUOTA 97.525.430/0001-69,VL_QUOTA 97.525.452/0001-29,VL_QUOTA 97.525.454/0001-18,VL_QUOTA 97.543.626/0001-86,VL_QUOTA 97.543.707/0001-86,VL_QUOTA 97.548.162/0001-09,VL_QUOTA 97.548.167/0001-23,VL_QUOTA 97.711.812/0001-87,VL_QUOTA 97.929.197/0001-80,VL_QUOTA 97.929.213/0001-34
DT_COMPTC,,,,,,,,,,,,,,,,,,,,,
2021-02-10,7157.0,305658.0,943.0,4.0,141.0,NaN,NaN,2731.0,196.0,40.0,...,2272.313422,2839.707202,148.975852,3289.709037,2.096676,NaN,3.537824,NaN,6.451052,9.535098
2021-02-11,7154.0,308296.0,943.0,4.0,141.0,NaN,NaN,2711.0,196.0,40.0,...,2288.293174,2865.372780,149.426837,3292.077749,2.114689,NaN,3.536515,NaN,6.541891,9.566748
2021-02-12,7152.0,310006.0,942.0,4.0,141.0,NaN,NaN,2709.0,196.0,40.0,...,2291.690487,2865.366965,149.592438,3292.496405,2.117458,NaN,3.537408,NaN,6.547986,9.585352


# Salva em pkl a base completa de fundos com + de 1 cotista

In [56]:
time_start = perf_counter()
vv.to_pickle(dir_fat_risco + 'fundos_mm_ready_to_go_20210223.pkl') 
time_end = perf_counter()
elapsed_time = time_end - time_start
print(' ****Salvar base COMPLETA de fundos MM com mais de 1 cotista demorou', round(elapsed_time,2) , ' segundos *****')


 ****Salvar base COMPLETA de fundos MM com mais de 1 cotista demorou 6.89  segundos *****
